In [70]:
import pandas as pd

columns = ["Delito", "Subdelito", "Fecha", "Victima", "Subvictima", "Adulto", "Genero", "Nacionalidad", "Provincia", "Canton", "Distrito"]

## Cargar los datos

In [71]:
police_15 = pd.read_csv('./Data/PJCROD_POLICIALES_V1-2015.csv',
                     encoding='latin1', names=columns, )

police_15['Fecha'] = pd.to_datetime(police_15['Fecha'])
police_15['Ano'] = police_15['Fecha'].apply(lambda x: x.year)

In [72]:
police_16 = pd.read_csv('./Data/PJCROD_POLICIALES_V1-2016.csv',
                     encoding='latin1', names=columns)

police_16['Fecha'] = pd.to_datetime(police_16['Fecha'])
police_16['Ano'] = police_16['Fecha'].apply(lambda x: x.year)

In [73]:
police_17 = pd.read_csv('./Data/PJCROD_POLICIALES_V1-2017.csv',
                     encoding='latin1', names=columns)

police_17['Fecha'] = pd.to_datetime(police_17['Fecha'])
police_17['Ano'] = police_17['Fecha'].apply(lambda x: x.year)

In [74]:
police_18 = pd.read_csv('./Data/PJCROD_POLICIALES_V1-2018.csv',
                     encoding='latin1', names=columns)

police_18['Fecha'] = pd.to_datetime(police_18['Fecha'])
police_18['Ano'] = police_18['Fecha'].apply(lambda x: x.year)

In [75]:
police_19 = pd.read_csv('./Data/PJCROD_POLICIALES_V1-2019.csv',
                     encoding='latin1', names=columns, index_col=False)

police_19['Fecha'] = pd.to_datetime(police_19['Fecha'])
police_19['Ano'] = police_19['Fecha'].apply(lambda x: x.year)

In [76]:
police = pd.concat([police_15,police_16,police_17,police_18,police_19])
police.sample(5)

,Delito,Subdelito,Fecha,Victima,Subvictima,Adulto,Genero,Nacionalidad,Provincia,Canton,Distrito,Ano
14170,HURTO,POR DESCUIDO,2016-04-02,VEHICULO,AUTOMOVIL [VEHICULO],Mayor de edad ...,HOMBRE,COSTA RICA,GUANACASTE,LIBERIA,LIBERIA,2016
48262,ROBO,CORTA CANDADOS,2016-10-29,EDIFICACION,TALLER [EDIFICACION],Mayor de edad ...,HOMBRE,COSTA RICA,PUNTARENAS,BUENOS AIRES,BUENOS AIRES,2016
15307,ROBO,FORZADURA,2018-04-01,EDIFICACION,OTRO O INDETERMINADO [EDIFICACION],Mayor de edad ...,HOMBRE,COSTA RICA,PUNTARENAS,GARABITO,TARCOLES,2018
7670,HOMICIDIO,AJUSTE DE CUENTAS/VENGANZA,2016-02-20,PERSONA,PEATON [PERSONA],Mayor de edad ...,HOMBRE,COSTA RICA,LIMON,MATINA,BATAN,2016
21625,ROBO,OTRO O INDETERMINADO,2019-05-18,PERSONA,OTRO O INDETERMINADO [PERSONA],Mayor de edad ...,HOMBRE,COSTA RICA,PUNTARENAS,GARABITO,JACO,2019


## Limpieza y Manipulacion de datos

In [77]:
# Cantones
police.loc[police['Canton'] == 'CAï¿½AS','Canton'] = 'CAÑAS'
police.loc[police['Canton'] == 'SARCHï¿½','Canton'] = 'SARCHI'

# Distritos
police.loc[police['Distrito'] == 'CAï¿½AS','Distrito'] = 'CAÑAS'
police.loc[police['Distrito'] == 'CAï¿½AS DULCES','Distrito'] = 'CAÑAS DULCES'
police.loc[police['Distrito'] == 'PEï¿½AS BLANCAS','Distrito'] = 'PEÑAS BLANCAS'
police.loc[police['Distrito'] == 'REVENTAZï¿½N','Distrito'] = 'REVENTAZON'

# Eliminar desconocido
desconocidos = ((police['Canton'] == 'DESCONOCIDO') |
                (police['Distrito'] == 'DESCONOCIDO') )
police.drop(police[desconocidos].index, inplace=True)

In [78]:
# Grecia --> Rio Cuarto
police.loc[police['Distrito'] == 'RIO CUARTO','Canton'] = 'RIO CUARTO'


In [79]:
police_grouped = police.groupby(['Delito', 'Ano', 'Provincia', 'Canton', 'Distrito']).size().to_frame('Count').reset_index()

In [92]:
poblaciones = pd.read_csv('./Data/Poblaciones.csv', encoding='latin1')
poblaciones.head()

,Provincia,Canton,Distrito,Poblacion
0,San Jose,San Jose,NaN,339581
1,San Jose,San Jose,Carmen,3070
2,San Jose,San Jose,Merced,14846
3,San Jose,San Jose,Hospital,23099
4,San Jose,San Jose,Catedral,15382


In [81]:
poblaciones['Provincia'] = poblaciones['Provincia'].str.upper()
poblaciones['Canton'] = poblaciones['Canton'].str.upper()
poblaciones['Distrito'] = poblaciones['Distrito'].str.upper()

In [82]:
police_final = police_grouped.merge(poblaciones, how='left',
                                    on=['Provincia','Canton', 'Distrito'])
police_final.head()


,Delito,Ano,Provincia,Canton,Distrito,Count,Poblacion
0,ASALTO,2015,ALAJUELA,ALAJUELA,ALAJUELA,442,47477.0
1,ASALTO,2015,ALAJUELA,ALAJUELA,CARRIZAL,5,8328.0
2,ASALTO,2015,ALAJUELA,ALAJUELA,DESAMPARADOS,42,31261.0
3,ASALTO,2015,ALAJUELA,ALAJUELA,GUACIMA,34,24598.0
4,ASALTO,2015,ALAJUELA,ALAJUELA,LA GARITA,12,8894.0


In [83]:
# Imprimir datos NA

# police_final[police_final["Poblacion"].isna()].to_excel('prueba.xlsx')

In [93]:
police_final.dropna(inplace = True)
police_final['Crimenes_1000'] = round((1000 * police_final['Count'])/police_final['Poblacion'],2)
police_final.to_csv('./Data/police_final.csv')

## Analisis Exploratorio


### Provincias

In [85]:
prov_pob = poblaciones.groupby('Provincia')['Poblacion'].sum().tolist()

info_provincia = police_final.groupby('Provincia')['Count'].sum()
info_provincia = info_provincia.reset_index()
info_provincia['Poblacion'] = prov_pob
info_provincia['Poblacion'] = info_provincia['Poblacion']/2
info_provincia

,Provincia,Count,Poblacion
0,ALAJUELA,44546,981681.0
1,CARTAGO,19292,529817.0
2,GUANACASTE,23033,377136.0
3,HEREDIA,26047,505052.0
4,LIMON,25998,444447.0
5,PUNTARENAS,34896,480548.0
6,SAN JOSE,106280,1621393.0


In [86]:
info_provincia['Crimenes_1000'] = round((1000 * info_provincia['Count'])/info_provincia['Poblacion'],2)
info_provincia.sort_values('Crimenes_1000')

,Provincia,Count,Poblacion,Crimenes_1000
1,CARTAGO,19292,529817.0,36.41
0,ALAJUELA,44546,981681.0,45.38
3,HEREDIA,26047,505052.0,51.57
4,LIMON,25998,444447.0,58.50
2,GUANACASTE,23033,377136.0,61.07
6,SAN JOSE,106280,1621393.0,65.55
5,PUNTARENAS,34896,480548.0,72.62


In [87]:
info_provincia_ano = police_final.pivot_table(values= 'Count',
                                              index= 'Provincia',
                                              columns= 'Ano',
                                              aggfunc= 'sum')
info_provincia_ano = info_provincia_ano.reset_index()
info_provincia_ano['Poblacion'] = prov_pob
info_provincia_ano['Poblacion'] = info_provincia_ano['Poblacion']/2
info_provincia_ano

Ano,Provincia,2015,2016,2017,2018,2019,Poblacion
0,ALAJUELA,8851,9086,8893,9308,8408,981681.0
1,CARTAGO,3954,3642,3671,4173,3852,529817.0
2,GUANACASTE,4889,4486,4695,4584,4379,377136.0
3,HEREDIA,4959,5398,5372,5565,4753,505052.0
4,LIMON,5035,5227,5008,5366,5362,444447.0
5,PUNTARENAS,6811,7403,6988,7034,6660,480548.0
6,SAN JOSE,18069,20911,22352,23013,21935,1621393.0


In [88]:
info_provincia_ano['Crimenes_1000_2015'] = round((1000 * info_provincia_ano[2015])/info_provincia_ano['Poblacion'],2)
info_provincia_ano['Crimenes_1000_2019'] = round((1000 * info_provincia_ano[2019])/info_provincia_ano['Poblacion'],2)
info_provincia_ano

Ano,Provincia,2015,2016,2017,2018,2019,Poblacion,Crimenes_1000_2015,Crimenes_1000_2019
0,ALAJUELA,8851,9086,8893,9308,8408,981681.0,9.02,8.56
1,CARTAGO,3954,3642,3671,4173,3852,529817.0,7.46,7.27
2,GUANACASTE,4889,4486,4695,4584,4379,377136.0,12.96,11.61
3,HEREDIA,4959,5398,5372,5565,4753,505052.0,9.82,9.41
4,LIMON,5035,5227,5008,5366,5362,444447.0,11.33,12.06
5,PUNTARENAS,6811,7403,6988,7034,6660,480548.0,14.17,13.86
6,SAN JOSE,18069,20911,22352,23013,21935,1621393.0,11.14,13.53


In [89]:
info_provincia_ano['Tasa'] = (info_provincia_ano['Crimenes_1000_2019']/info_provincia_ano['Crimenes_1000_2015']-1)*100
info_provincia_ano

Ano,Provincia,2015,2016,2017,2018,2019,Poblacion,Crimenes_1000_2015,Crimenes_1000_2019,Tasa
0,ALAJUELA,8851,9086,8893,9308,8408,981681.0,9.02,8.56,-5.099778
1,CARTAGO,3954,3642,3671,4173,3852,529817.0,7.46,7.27,-2.546917
2,GUANACASTE,4889,4486,4695,4584,4379,377136.0,12.96,11.61,-10.416667
3,HEREDIA,4959,5398,5372,5565,4753,505052.0,9.82,9.41,-4.175153
4,LIMON,5035,5227,5008,5366,5362,444447.0,11.33,12.06,6.443071
5,PUNTARENAS,6811,7403,6988,7034,6660,480548.0,14.17,13.86,-2.187721
6,SAN JOSE,18069,20911,22352,23013,21935,1621393.0,11.14,13.53,21.454219


In [91]:
prov_delito_ano = police_final.pivot_table(values= 'Count',
                                              index= ['Provincia','Delito'],
                                              columns= 'Ano',
                                              aggfunc= 'sum')
prov_delito_ano.reset_index(inplace=True)

In [ ]:
tabla_poblaciones = poblaciones.groupby('Provincia')['Poblacion'].sum().reset_index()

In [ ]:
prov_delito_ano = prov_delito_ano.join(tabla_poblaciones.set_index('Provincia'), on='Provincia')

In [ ]:
prov_delito_ano['Poblacion'] = prov_delito_ano['Poblacion']/2

In [ ]:
prov_delito_ano['Crimenes_1000_2015'] = round((1000 * prov_delito_ano[2015])/prov_delito_ano['Poblacion'],6)
prov_delito_ano['Crimenes_1000_2019'] = round((1000 * prov_delito_ano[2019])/prov_delito_ano['Poblacion'],6)
prov_delito_ano['Tasa'] = (prov_delito_ano['Crimenes_1000_2019']/prov_delito_ano['Crimenes_1000_2015']-1)*100

In [ ]:
prov_delito_ano[prov_delito_ano['Provincia'] == 'SAN JOSE'].sort_values('Tasa', ascending=False).head(50)


### Cantones


In [ ]:
cant_pob = poblaciones.groupby('Canton')['Poblacion'].sum().reset_index()
cant_pob

In [ ]:
info_canton = police_final.pivot_table(values= 'Count',
                                      index= ['Provincia','Canton'],
                                      columns= 'Ano',
                                      aggfunc= 'sum')
info_canton.reset_index(inplace=True)

info_canton = info_canton.join(cant_pob.set_index('Canton'), on='Canton')
info_canton['Poblacion'] = info_canton['Poblacion']/2

media_poblacional = info_canton.loc[:,[2015,2016,2017,2018,2019]].mean(axis=1)
info_canton['Media'] = (1000* media_poblacional)/info_canton['Poblacion']

info_canton

In [99]:
info_canton['Crimenes_1000_2015'] = round((1000 * info_canton[2015])/info_canton['Poblacion'],2)
info_canton['Crimenes_1000_2019'] = round((1000 * info_canton[2019])/info_canton['Poblacion'],2)
info_canton['Tasa'] = (info_canton['Crimenes_1000_2019']/info_canton['Crimenes_1000_2015']-1)*100
info_canton

,Provincia,Canton,2015,2016,2017,2018,2019,Poblacion,Media,Crimenes_1000_2015,Crimenes_1000_2019,Tasa
0,ALAJUELA,ALAJUELA,3135,3291,3535,3711,3071,302074.0,11.085363,10.38,10.17,-2.023121
1,ALAJUELA,ATENAS,197,201,208,259,225,28424.0,7.669575,6.93,7.92,14.285714
2,ALAJUELA,GRECIA,462,524,478,529,450,75383.0,6.481567,6.13,5.97,-2.610114
3,ALAJUELA,GUATUSO,147,114,83,110,130,18541.0,6.299552,7.93,7.01,-11.601513
4,ALAJUELA,LOS CHILES,273,236,289,280,352,31546.0,9.066126,8.65,11.16,29.017341
...,...,...,...,...,...,...,...,...,...,...,...,...
77,SAN JOSE,SANTA ANA,446,573,456,542,570,58159.0,8.896302,7.67,9.80,27.770535
78,SAN JOSE,TARRAZU,82,75,56,69,92,18102.0,4.132140,4.53,5.08,12.141280
79,SAN JOSE,TIBAS,749,831,866,912,659,82889.0,9.692480,9.04,7.95,-12.057522
80,SAN JOSE,TURRUBARES,33,48,28,42,53,6619.0,6.164073,4.99,8.01,60.521042


In [107]:
#info_canton.to_csv('./Data/info_canton.csv')
police_final.groupby(['Canton', 'Distrito', 'Ano'], as_index=False)['Count'].sum()

0        True
1        True
2        True
3        True
4        True
        ...  
2316    False
2317    False
2318    False
2319    False
2320    False
Name: Canton, Length: 2321, dtype: bool